Sentinel NDVI calculation for Ivars Reinfield

In [1]:
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd
import dask
import datacube 
from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
from datacube.utils import geometry

import fiona
import rasterio.features
from osgeo import gdal, ogr
import os
from rsgislib.segmentation import segutils
from rasterstats import zonal_stats

#import custom functions
import sys
sys.path.append('src')
import DEAPlotting, SpatialTools, BandIndices, DEADataHandling
from load_data import load_data
from transform_tuple import transform_tuple
from imageSeg import imageSeg
from query_from_shp import query_from_shp

In [2]:
# where is your data and results folder?
data = 'data/'
results = 'results/'
shp_fpath = shp_fpath = 'data/spatial/murrumbidgee_catchment.shp'

#Input your area of interest's name, coords, and 
#the year you're interested in?
AOI = 'training'
year = 'winter'

time_period = ('2016-04-01', '2016-05-31')

#What thresholds should I use?
threshold = 0.8
wofs_theshold = 0.15

In [3]:
#Creating a folder to keep things neat
directory = results + AOI + "_" + year
if not os.path.exists(directory):
    os.mkdir(directory)

results = results + AOI + "_" + year + "/"

In [ ]:
query = query_from_shp(shp_fpath, time_period[0], time_period[1], dask_chunks = 0)
query['resolution'] = (-25,25)
query['output_crs'] = ('epsg:3577')

dc = datacube.Datacube(app='load_clearsentinel')
#landsat
landsat = DEADataHandling.load_clearlandsat(dc=dc, query=query,sensors=['ls8'], 
                        bands_of_interest=['red', 'green', 'blue'],
                        masked_prop=0.2, mask_pixel_quality=True)

with fiona.open(shp_fpath) as shapes:
        crs = geometry.CRS(shapes.crs_wkt)
        first_geometry = next(iter(shapes))['geometry']
        geom = geometry.Geometry(first_geometry, crs=crs)

mask = rasterio.features.geometry_mask([geom.to_crs(landsat.geobox.crs) for geoms in [geom]],
                                           out_shape=landsat.geobox.shape,
                                           transform=landsat.geobox.affine,
                                           all_touched=False,
                                           invert=True)
# Mask the xarrays
landsat = landsat.where(mask)

# dc = datacube.Datacube(app='wofs')
# del query['time']
# wofs_alltime = dc.load(product = 'wofs_summary', **query)

Loading ls8 pixel quality
Loading ls8 pixel quality
    Skipping ls8; no valid data for query
Combining and sorting  data


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



In [ ]:
#band indices calculation
def ndvi_func(nir, red):
    return ((nir - red)/(nir + red))

def ndvi_ufunc(ds):
    return xr.apply_ufunc(
        ndvi_func, ds.nbart_nir_1, ds.nbart_red,
        dask='parallelized',
        output_dtypes=[float])

NDVI_sentinel = ndvi_ufunc(sentinel)
NDVI_max = NDVI_sentinel.groupby('x','y').max('time').rename('NDVI_max')
NDVI_std = NDVI_sentinel.groupby('x','y').std('time').rename('NDVI_max')

In [ ]:
transform, projection = transform_tuple(NDVI_max, (NDVI_max.x, NDVI_max.y), epsg=3577)
SpatialTools.array_to_geotiff(results + AOI + "_" + year + ".tif",
              NDVI_max.values, geo_transform = transform, 
              projection = projection, nodata_val=0)

In [ ]:
# setup input filename
InputNDVIStats = results + AOI + "_" + year + ".tif"
KEAFile = results + AOI + '_' + year + '.kea'
SegmentedKEAFile = results + AOI + '_' + year + '_sheperdSEG.kea'
SegmentedTiffFile = results + AOI + '_' + year + '_sheperdSEG.tif'
SegmentedPolygons = results + AOI + '_' + year + '_SEGpolygons.shp'
print("calculating imageSegmentation")
imageSeg(InputNDVIStats, KEAFile, SegmentedKEAFile, SegmentedTiffFile, SegmentedPolygons, minPxls = 100)

In [ ]:
gdf = gpd.read_file(results + AOI + '_' + year + '_SEGpolygons.shp')
#calculate zonal mean of NDVI
gdf['mean'] = pd.DataFrame(zonal_stats(vectors=gdf['geometry'], raster=InputNDVIStats, stats='mean'))['mean']
#calculate area of polygons
gdf['area'] = gdf['geometry'].area
#filter by area and mean NDVI
highNDVI = gdf['mean'] >= threshold
smallArea = gdf['area'] <= 5500000
gdf = gdf[highNDVI & smallArea]
#export shapefile
gdf.to_file(results + AOI + "_" + year + "_Irrigated.shp")

In [ ]:
#get the transform and projection of our gtiff
transform, projection = transform_tuple(NDVI_max, (NDVI_max.x, NDVI_max.y), epsg=3577)
#find the width and height of the xarray dataset we want to mask
width,height = NDVI_max.shape
# rasterize vector
gdf_raster = SpatialTools.rasterize_vector(results + AOI + "_" + year + "_Irrigated.shp",
                                           height, width, transform, projection, raster_path=None)
# Mask the xarray
NDVI_max_Irrigated = NDVI_max.where(gdf_raster)

In [ ]:
#remove areas below our threshold that are at the edges of the rasterized polygons
NDVI_max_Irrigated = NDVI_max_Irrigated.where(NDVI_max_Irrigated >= threshold)
#Use wofs to remove areas that have standing water for a significant amount of time
NDVI_max_Irrigated = NDVI_max_Irrigated.where(wofs_alltime.frequency.drop('time').squeeze() <= wofs_theshold)

#remove pixels that cross over the major rivers in the region
rivers_raster = SpatialTools.rasterize_vector("data/spatial/major_rivers_aus.shp", height, width, transform, projection, raster_path=None)
rivers_raster = rivers_raster.astype(bool)
rivers_raster = xr.DataArray(rivers_raster, coords = [NDVI_max.y, NDVI_max.x], dims = ['y', 'x'], name='rivers')
NDVI_max_Irrigated = NDVI_max_Irrigated.where(rivers_raster == 0)

In [ ]:
NDVI_max_Irrigated.plot(figsize=(10,10))

In [ ]:
ones = np.count_nonzero(~np.isnan(NDVI_max_Irrigated.values))
area = (ones*(25*25)) / 1000000
print("Around " + AOI + " during " + str(year) + ", " + str(area) + " km2 was under irrigated cultivation")

In [ ]:
a = xr.open_rasterio("results/Peel_s2_Summer2018-19/Peel_s2_Summer2018-19_10m_600pixels_Irrigated.tif")

In [ ]:
ones = np.count_nonzero(~np.isnan(a.values))
area = (ones*(10*10)) / 1000000
print("Around " + AOI + " during " + str(year) + ", " + str(area) + " km2 was under irrigated cultivation")

In [ ]:
SpatialTools.array_to_geotiff(results + AOI + "_" + year + "_25m_100pixels_Irrigated.tif",
              NDVI_max_Irrigated.values,
              geo_transform = transform, 
              projection = projection, 
              nodata_val=0)